# Introduction to Ribo-seq data analysis

## Sections:
   - 1.1 Short introduction to Jupyter (Notebook) and the JupyterHub
   - 1.2 Overview of the `rpbp` pipeline: command line options and configuration file
   - 1.3 High-level introduction to Ribo-seq, de novo ORF discovery (elements of annotation, transcript isoforms, CDS, UTRs, etc.), biological relevance of alternative translation events (including translation from non-coding transcripts), and why we need "dedicated software" to analyse Ribo-seq data.
   - 1.4 Ribo-seq quality control how-to.
     
## Questions & Objectives:
   - Jupyter Notebook basics: dashboard, user interface, navigation, running code, etc. How to use Jupyter with virtual environments.
   - How to use the JupyterHub.
   - What is the translatome? What are the uses of Ribo-seq.
   - Why do we need dedicated software to analyse Ribo-seq data? What softwares are available to analyse Ribo-seq data?
   - Run one such software, `rpbp`, on a selected Ribo-seq dataset.
   - Learn how to identify "good quality" Ribo-seq data.
   - Learn how to visualise the results.
     
### After I will be able to:
   - Understand what is a Jupyter Notebook, and how to use it.
   - Understand how to analyse Ribo-seq data for ORF discovery.
   - Use the ORF predictions for follow-up studies.


## 1.1 Short introduction to Jupyter (Notebook) and the JupyterHub

The Jupyter Notebook is an open-source web-based application allowing in-browser editing (Jupyter is running on your own computer, *i.e* your computer acts as the server), combining text, code, computations and rich media output (notebook documents). Jupyter supports many programming languages, including Python, R, and Julia. The JupyterHub is a multi-user version of the Notebook (this notebook is served via "our Hub" at https://jupyter.internal).

### Basic workflow
***

Typically, a notebook document is organised into cells, and one moves forward from one cell to the next, breaking the content or the computation into separate parts. This workflow allows to "validate" the output of one cell before moving to the next, and is also convenient for interactive exploration, documentation, *etc.*

### Jupyter Notebook basics
***

The Notebook dashboard: when you first start the notebook server, your browser will open to the notebook dashboard. This exact appearance of the dashboard may vary depending on the version. To create a new notebook, *e.g.* click on the *File > New > Notebook* and select a kernel from the Kernel dropdown menu or tab. Check the top menu and tool bar (see at the top of this notebook!) 

Modal editor: **edit mode**. Edit mode is indicated by a green cell border and a prompt showing in the editor area. When a cell is in edit mode, you can type into the cell, like a normal text editor. Enter edit mode by pressing `Enter` or using the mouse to click on a cell.

![](img/jupyter-1.png)

Modal editor: **command mode**. Command mode is indicated by a grey cell border with a blue left margin. When you are in command mode, you are able to edit the notebook as a whole, but not type into individual cells (be careful to type into a cell in command mode!). Enter command mode by pressing `Esc` or using the mouse to click outside a cell.
    
![](img/jupyter-2.png)


### Basic commands
***

Edit mode

| Command | action |
|------|------|
|   `Ctrl-Enter`  | run selected cells |
|   `Shift-Enter`  | run cell, select below |<br>

Command mode

| Command | action |
|------|------|
|   `Enter`  | enter edit mode |
|   `Esc`  | enter command mode |
|   `a`  | insert cell above |
|   `b`  | insert cell below |

## More
***

- See [Project Jupyter](https://jupyter.org/) for installation instructions, detailed documentation, *etc.*
- After the course, explore the menu (Help) of this notebook, and experiment with basic commands.

## 1.2 Overview of the rpbp pipeline: command line options and configuration file

For practical reasons, we will cover some technical aspects before introducing ribosome-profiling in more details. In particular, we will first set-up the files to run `rpbp` on the example Ribo-seq dataset, briefly introduce the `Slurm` workload manager, and actually run the `rpbp` pipeline. While our data is running, we will then go into more details about Ribo-seq, the `rpbp` package and some methodological aspects behind it.

<div class="alert alert-block alert-warning"><b>Dependencies</b>
    
All the core software that will be used in this part of the lecture is already available on the HPC cluster. A [conda](https://docs.conda.io/projects/conda/en/latest/index.html) environment has been setup with all required packages. To perform the analyses for this part of the practical, you will need to activate the environment as follows:

```bash
# adjust year e.g. bioinfo_2025_course
conda activate /biosw/bioinfo_2025_course/conda/transcriptomics
```
</br>

You will also need to create a <i>kernel spec</i> to use this environment with this notebook. To do so

```bash
python -m ipykernel install --user --name="transcriptomics-python"
```
</br>
then switch to the <i>transcriptomics-python</i> kernel (menu bar <i>Kernel > Change kernel...</i> or top right corner <i>Switch kernel</i>).
</div>


The `rpbp` pipeline consists of an index creation step, which must be performed once for each genome and set of annotations, and a two-phase prediction pipeline, which must be performed for each sample. In the first phase of the prediction pipeline, ORF profiles are created. In the second phase, the ORFs which show evidence of translation are identified.

### Creating reference genome indices
***

The entire index creation process can be run automatically using the following command:

`
prepare-rpbp-genome <config> [options]
`

See [How to prepare genome indices and annotations](https://rp-bp.readthedocs.io/en/latest/user-guide.html#how-to-prepare-genome-indices-and-annotations) for detailed information. To save time, we have already created the indices for the human genome (GRCh38.96). 


### Running the pipeline
***

The entire `rpbp` pipeline (2 steps) can be run on a set of riboseq samples, including any biological replicates. To run the pipeline, we first need to prepare a configuration file, consisting in a series of required (and optional) key: value pairs. We will explain this below.

Lastly, to run the pipeline on the cluster, we will use the `Slurm` workload manager. `Slurm` is a job scheduler. For our purpose, it suffices to know that it provides a framework to ask for resources and execute our job on the cluster. To submit our job, we will use the command `sbatch`, and to monitor the status of our job, we will use `squeue -u username`. 
See online for general information on [How to run the pipeline](https://rp-bp.readthedocs.io/en/latest/user-guide.html#how-to-run-the-pipeline).



***
<div class="alert alert-block alert-danger"><b>The cells below contain "code": run them one after the other!</b></div>


In [7]:
# import modules that are needed to run this notebook
from pathlib import Path

# some functions, definitions, etc. that are needed to run this notebook
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writefile_globals(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

We assume that the repository has been cloned to your `$HOME` directory.

In [16]:
# Prepare to run the example (downsampled) dataset: 4 replicates, 2 PBS, 2 EGF.
# set-up directory structure for the analysis
PARENT = Path(Path.home(), 'bioinformatics-lectures', 'transcriptomics', 'Riboseq')
DIRLOC = Path(PARENT, 'local', 'downsampled-analysis')
CFG = Path(DIRLOC, 'config')
CFG.mkdir(parents=True, exist_ok=True)
RES = Path(DIRLOC, 'riboseq-results')
RES.mkdir(parents=True, exist_ok=True)
# change directory
%cd $CFG

/beegfs/pub/bioinformatics-lectures/transcriptomics/Riboseq/local/downsampled-analysis/config


We first need to prepare a YAML configuration file to run the rpbp package. YAML ("YAML Ain't Markup Language") is a data-serialization (markup) language.

In [20]:
%%writefile_globals downsampled.yaml

# check https://rp-bp.readthedocs.io/en/latest/user-guide.html#how-to-prepare-the-configuration-file

project_name: downsampled

# location of existing index files and annotations
genome_base_path: /pub/riboseq-index/GRCh38_96
genome_name: GRCh38.96
gtf: /pub/riboseq-index/GRCh38_96/GRCh38.96.gtf
fasta: /pub/riboseq-index/GRCh38_96/GRCh38_96.fa
star_index: /pub/riboseq-index/GRCh38_96/star
ribosomal_index: /pub/riboseq-index/rRNA_cluster_mt_rRNA/rRNA_cluster_mt_rRNA
ribosomal_fasta: /pub/riboseq-index/rRNA_cluster_mt_rRNA/rRNA_cluster_mt_rRNA.fasta
adapter_file: /pub/riboseq-index/riboseq-adapters.fa

# The base location for the created files
riboseq_data: {RES}

riboseq_samples:
 dSRR7451194.EGF.rep-1: /pub/raw-data/downsampled/dSRR7451194_1.fastq.gz
 dSRR7451184.EGF.rep-2: /pub/raw-data/downsampled/dSRR7451184_1.fastq.gz
 dSRR7451191.PBS.rep-1: /pub/raw-data/downsampled/dSRR7451191_1.fastq.gz
 dSRR7451197.PBS.rep-2: /pub/raw-data/downsampled/dSRR7451197_1.fastq.gz

riboseq_biological_replicates:
 EGF:
  - dSRR7451194.EGF.rep-1
  - dSRR7451184.EGF.rep-2
 PBS:
  - dSRR7451191.PBS.rep-1
  - dSRR7451197.PBS.rep-2

riboseq_sample_name_map:
 dSRR7451194.EGF.rep-1: EGF1
 dSRR7451184.EGF.rep-2: EGF2
 dSRR7451191.PBS.rep-1: PBS1
 dSRR7451197.PBS.rep-2: PBS2

# Rp-Bp options: we need to change the default parameters to run the downsampled data.
# Generally, you do not need to change the default parameters!

# The number of bases upstream of the translation initiation site to begin 
# constructing the metagene profile.
metagene_start_upstream: 50
# The number of bases downstream of the translation initiation site to end 
# the metagene profile.
metagene_start_downstream: 50
# The number of bases upstream of the translation termination site to begin 
# constructing the metagene profile.
metagene_end_upstream: 50
# The number of bases downstream of the translation termination site to end 
# the metagene profile.
metagene_end_downstream: 50

# N.B. These values are set artificially low for the example to work!
min_metagene_profile_count: 50
min_metagene_image_count: 10

# N.B. These value are set low to reduce the running time, but will affect the results.
metagene_iterations: 100
translation_iterations: 100


Write the submission script and change permission by running the next cells:

In [18]:
%%writefile_globals runjob-downsampled
#!/bin/bash
#SBATCH -J "practical"
#SBATCH -n 1
#SBATCH -N 1
#SBATCH -c 1
#SBATCH --mem=20G

# call rpbp
run-all-rpbp-instances downsampled.yaml \
    --merge-replicates \
    --run-replicates \
    --keep-intermediate-files \
    --num-cpus 12 \
    --mem 120G \
    --use-slurm \
    --logging-level INFO \
    --log-file downsampled.log

In [ ]:
!chmod 777 runjob-downsampled

***
<div class="alert alert-block alert-danger"><b>Commands must now be run on the command line!</b>

Go to your terminal, navigate to <i>bioinformatics-lectures/transcriptomics/Riboseq/local/downsampled-analysis/config</i>, and

```bash
sbatch runjob-downsampled
```
</br>

You can check the status of the job by issuing 


```bash
squeue
```
</br>
and/or by looking at the log files (<i>downsampled.log</i> and <i>slurm</i> files).</br>
Results will be written to a directory named <i>bioinformatics-lectures/transcriptomics/Riboseq/local/downsampled-analysis/riboseq-results</i>.

</div>


***
  
<div class="alert alert-block alert-success"><b>Hints</b>
    
How to run the pipeline from your computer?
</br>
The `rpbp` pipeline can easily be installed from [bioconda](http://bioconda.github.io/) or you can use containers, see [Installation](https://rp-bp.readthedocs.io/en/latest/installation.html#installation). You can then use the same command that we used above, omitting all `Slurm`-related headers and command options.
</div>





## 1.3 High-level introduction to Ribo-seq

The idea behind the Ribo-seq protocol can be summarised in a few lines: ribosome-protected fragments or footprints (RNA protected by the ribosome), can be isolated through the use of nucleases that degrade unprotected RNA regions, and submitted to a deep sequencing protocol similar to those used for RNA. In brief, *(i)* ribosome-bound RNA is isolated from cell/tissue lysates; *(ii)* treated with a drug, depending on the purpose of the experiment, *e.g* elongating (cycloheximide, but can distort ribosome profiles as initiation can continue even though elongation is blocked, which can lead to high ribosome density at alternative initiation sites) or initiating ribosomes (harringtonine, lactimidomycin/puromycin), for eukaryotes, the choice of inhibitor has a concentration-dependent impact on the kinetics of initiation and elongation; *(iii)* nucleases are added to digest the unprotected RNA (RNase), the choice of nuclease and treatment strongly affect the ribosome profiles; *(iv)* after footprint recovery, rRNA depletion is performed and samples are sequenced, but protocols vary a lot (circularisation step or no, bias reduction methods, *etc*), and there are always new methods! ([Ligation-free ribosome profiling](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-1005-1), [RiboLace](https://www.sciencedirect.com/science/article/pii/S2211124718315444?via%3Dihub), to name but a few) 

By mapping the position of translating ribosomes over the entire transcriptome, Ribo-seq provides a snapshot of the entire translatome. Ribo-seq has been used to answer a wide range of questions, including identification of translated small open reading frames (ORFs), non-coding sequences and alternative reading frames, quantification of translational control (in combination with RNA-seq, *i.e* translational efficiency, or regulatory mechanisms associated with the translation process itself, such as upstream ORFs acting in cis, or translation regulating transcript stability by triggering NMD via recognition of a premature stop codon), or for gaining mechanistic insights on the translation process itself.

### Open reading frame discovery
***

We understand an open reading frame (ORF) as a potentially translatable sequence that consists of a series of codons beginning with a start codon and ending with a stop codon. Translatable ORFs can be found anywhere: in the 5' untranslated region (5'UTR), in the 3' untranslated region (3'UTR), within or overlapping with annotated coding sequences (CDSs), in transcripts that were previously thought to be non-coding (lincRNAs, antisense, pseudogene, or other processed transcripts), or in novel transcripts (intra/intergenic).

![](img/ribo-1.png)

As we will briefly explain below, in the `rpbp` workflow, ORFs are labeled according to their position and exon structure relative to the annotations. Except for annotated CDSs (Canonical), the assignment of these labels depend on the complexity of the annotations, and thus are not "fixed".


| Label | Description |
|------|------|
|   **Canonical**  |  An ORF that coincides with an annotated coding sequence (CDS from a protein coding transcript) |
|   **Can. (variant)**  | An ORF that is in-frame (with respect to a CDS), N-terminus extended or truncated |
|   **uORF** or five prime |  An ORF that is in the 5'UTR of a CDS and do not overlap other CDS on the same strand (from alternative transcripts) |
|   **dORF** or three prime | An ORF that is in the 3'UTR of a CDS and do not overlap other CDS on the same strand (from alternative transcripts) |
|   **ncORF** or noncoding | An ORF that originates from a transcript not annotated as coding (non-coding, processed transcript or pseudogene) |

An overlapping uORF is an out-of-frame ORF overlapping on the same transcript both the 5'UTR and the canonical coding sequence. An overlapping dORF is an out-of-frame ORF overlapping on the same transcript both the 3'UTR and the canonical coding sequence. 

#### Small open reading frames and biological relevance of alternative translation events
***

We refer to a small ORF (**sORF**) as an ORF that contains less than 100 amino acids. 

Accumulated evidence from Ribo-seq (and proteomics) experiments over the last 5 years suggests that small open reading frames-encoded peptides are underpredicted, even for well-annotated model species. sORFs can be found over a wide variety of transcripts. So far, a number of sORFs encoding functional peptides have been identified, such as Sarcolamban, Myoregulin, Myomixer, Minion, MOXI, MOTS-c, SPAR, or NoBody, to name but a few.


Tools such as `rpbp` can be used as part of a *sORF discovery workflow*, as depicted below:

<br>
<br>
<div>
<img src="img/ribo-2.png" width="800" align="center"/>
</div>


### More
***

Many recent publications address the role of alternative translation events:
- [Standardized annotation of translated open reading frames](https://www.nature.com/articles/s41587-022-01369-0)
- [A high-resolution map of human RNA translation](https://www.sciencedirect.com/science/article/pii/S1097276522006062?via%3Dihub)
- [The Translational Landscape of the Human Heart](https://www.sciencedirect.com/science/article/pii/S0092867419305082)
- [Translation of Small Open Reading Frames: Roles in Regulation and Evolutionary Innovation](https://www.sciencedirect.com/science/article/pii/S0168952518302221)



### Why do we need "dedicated software" to analyse Ribo-seq data?
***

<br>
<div>
<img src="img/ribo-3.png" width="700" align="center"/>
</div>


Conceptually, actively translated ORFs are predicted based on ribosome density. In general, however, ribosome occupancy itself, as indicated by the fragment reads mapped on the transcriptome, is not sufficient (due to noise from ligation bias in cloning ribosome footprints and amplification by PCR, regulatory RNAs that bind with the ribosome, ribosome engagement without translation, *etc.*). The technique allows to recover the precise location of the peptidyl-site (P-site) of the ribosome within each fragment or read, owing to its single-nucleotide resolution. Aligned by their P-site positions, the reads from actively translating ribosomes should therefore exhibit a 3-nt periodicity along the ORF. To select reads and predict translation, `rpbp` takes advantage of this 3-nt periodicity.

### More
***

- [Comparison of software packages for detecting unannotated translated small open reading frames by Ribo-seq](https://doi.org/10.1093/bib/bbae268)


## 1.4 Ribo-seq quality control how-to

Depending on the efficiency of the rRNA removal step of the experimental protocol, small structured RNAs (rRNAs, tRNAs, or snoRNAs) may have to be removed in a first pre-processing step. The remaining reads are mapped using splice-aware aligner (`STAR`), but reads can still map to multiple locations. Different strategies can be implemented to either rescue the reads, keep one primary alignment per read, or discard all multi-mapping reads altogether. Finally, only periodic reads are kept for the analysis. We will use the data from the output of `Flexbar` (reads filtered for quality, trimmed from adapters), `Bowtie2` (reads mapping to rRNA, clean reads), `STAR` (unique reads) and `rpbp` (periodic reads) to quantify the amount of reads filtered out at each step of the pipeline.

In a high-quality Ribo-seq library, reads mostly map to coding sequences (CDS or Canonical) (typically >85%) and to the 5'UTR (up to 10%). A smaller proportion map to the 3'UTR. The amount of reads mapping to non-coding regions can vary, but in general the signal is not very strong. Using the results of the pipeline, we will explore how many ORFs are predicted in each regions. It is also possible to use other tools such as `bedtools coverage`, that uses the mapped data (before translation prediction).

A characteristic feature of a high-quality Ribo-seq library is its read-length distribution, which typically peaks around 29 nt in eukaryotic organisms, however broader distributions can be observed under different protocols, depending on the nuclease treatment, the drugs/inhibitors used, *etc.* It is also known that different ribosomal conformation correspond to distinct read-length distributions, and that these can also be affected by ribosomes belonging to different pools (mitochondrial ribosomes were shown previously to display a bimodal distribution, compared to cytosolic-derived fragments). All these considerations must be taken into account when analysing the distribution of read lengths.

We will briefly explore these and other aspects graphically, using the data from the example. 

### How to summarize the results

***
<div class="alert alert-block alert-danger"><b>Commands must now be run on the command line!</b>

Go to your terminal and navigate to <i>bioinformatics-lectures/transcriptomics/Riboseq/local/downsampled-analysis/config</i>.
To prepare the summary output for the profile construction dashboard, run the following

```bash
srun -c 4 summarize-rpbp-profile-construction downsampled.yaml --num-cpus 4 --logging-level INFO --log-file rpbp-profile-summary.log &
```
</br>

and for the predictions dashboard

```bash
srun summarize-rpbp-predictions downsampled.yaml --no-replicates --logging-level INFO --log-file rpbp-predictions-summary.log &
```
</br>

When jobs are completed, you can launch the apps to visualize and explore the results. Launch any of the web applications with

```bash
rpbp-profile-construction-dashboard --host 0.0.0.0 -c downsampled.yaml
```
</br>

or 

```bash
rpbp-predictions-dashboard --host 0.0.0.0 -c downsampled.yaml
```
</div>


***

<div class="alert alert-block alert-info"><b>Self-guided learning </b>
    

As a self-guided learning exercise, you can try to explore results from the full dataset, available under */pub/bioinformatics-lectures/transcriptomics/Riboseq/local/full-analysis*</div>


***

### Explore results in more details

In the paper, they show how EGF stimulation results in distinct "waves" of protein synthesis. Proteins whose synthesis was statistically significantly altered in at least one time point were subjected to hierarchical clustering.

We pick 2 genes from cluster 1 (characterized by increased protein synthesis as early as 30 min following EGF stimulation): EGR1 (refered to as a canonical immediate early gene or IEG, *i.e* a gene that is rapidly up-regulated within the first hour following stimulus) and ATF3 (a "classical" delayed early gene; in contrast to established literature, this one appears to be rapidly down-regulated after 15 min following EGF stimulation, then up-regulated after 30 min).


***
<div class="alert alert-block alert-danger"><b>The cells below contain "code": run them one after the other!</b></div>

In [24]:
import pbiotools.utils.bed_utils as bed_utils

full_path = Path('/pub', 'bioinformatics-lectures', 'transcriptomics', 'Riboseq', 'local', 'full-analysis')
orf_file = Path(full_path, 'riboseq-results', 'analysis', 'rpbp_predictions', 'full-unique.filtered.predicted-orfs.bed.gz')
predicted_orfs = bed_utils.read_bed(orf_file)


In [33]:
mEGR1 = predicted_orfs['transcripts'].str.contains('ENST00000239938') # one EGR1 transcript
predicted_orfs[mEGR1][['seqname',
 'start',
 'end',
 'id',
 'orf_len',
 'orf_type',
 'condition',
 'x_1_sum',
 'x_2_sum',
 'x_3_sum']]


,seqname,start,end,id,orf_len,orf_type,condition,x_1_sum,x_2_sum,x_3_sum
3343,5,138467199,138468078,ENST00000239938_5:138467199-138468078:+,879,altCDS,dSRR7451184.EGF.rep-2,7.0,1.0,3.0
3344,5,138467568,138468078,ENST00000239938_5:138467568-138468078:+,510,altCDS,dSRR7451194.EGF.rep-1,6.0,4.0,1.0


In [38]:
mATF3 = predicted_orfs['transcripts'].str.contains('ENST00000492118') # one ATF3 transcript
predicted_orfs[mATF3][['seqname',
 'start',
 'end',
 'id',
 'orf_len',
 'orf_type',
 'condition',
 'x_1_sum',
 'x_2_sum',
 'x_3_sum']]

,seqname,start,end,id,orf_len,orf_type,condition,x_1_sum,x_2_sum,x_3_sum


What do you notice from the output above?

<br>
<br>
<div>
<img src="img/Human_ATF3.png" width="900" align="center"/>
</div>
<br>
<br>

ATF3 is a gene encoding an activation transcription factor. This gene is induced by a variety of signals, including cellular stress response. 

Another member of this family is ATF4. The translation of ATF4 is known to be dependent on uORFs. In particular, during normal conditions, uORFs repress translation of the main ATF4 CDS, while under stress conditions ATF4 is translated.

What do you notice from the output below?

In [35]:
mATF4 = predicted_orfs['transcripts'].str.contains('ENST00000404241') # one ATF4 transcript
predicted_orfs[mATF4][['seqname',
 'start',
 'end',
 'id',
 'orf_len',
 'orf_type',
 'condition',
 'x_1_sum',
 'x_2_sum',
 'x_3_sum']]

,seqname,start,end,id,orf_len,orf_type,condition,x_1_sum,x_2_sum,x_3_sum



***

AGPL-3.0 (code and scripts)

Copyright (c) 2025 Etienne Boileau